In [60]:
from random import randint
import random
import numpy as np
import pandas as pd
import random
from openpyxl import load_workbook

wb = load_workbook(filename= 'Dataset.xlsx')
sheet_ranges = wb['Sheet 1']

df = pd.DataFrame(sheet_ranges.values)
df

col = df[1:72][[4,5,6,7,9,10,11,12,13]]
col.columns = ['UAW','S','A','C','UUCW','TCF','ECF','Real','Real_Effort']
col1 = pd.DataFrame(col)

col1.insert(9,"Desired_UUCW", 0 , True)

for index, row in col1.iterrows():
    UAW = col1.loc[index,"UAW"]
    UUCW = col1.loc[index,"UUCW"]
    TCF = col1.loc[index,"TCF"]
    ECF = col1.loc[index,"ECF"]
    Real_E = col1.loc[index,"Real_Effort"]
    
    desired_UUCW = Real_E/((TCF*ECF)*20)-UAW
    #col1.at(5,"UUCW Algen", desired_UUCW , True)
    col1.at[index, 'Desired_UUCW'] = np.round(desired_UUCW,1)


col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,576
2,8,4,20,15,445,0.99,0.99,398.1,7962,398
3,9,1,5,20,355,1.03,0.8,396.75,7935,472
4,8,5,10,15,350,0.9,0.91,390.25,7805,468
5,8,1,10,16,345,0.9,0.91,387.9,7758,465
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,272
68,17,1,23,22,565,1.03,0.8,289.55,5791,334
69,19,5,30,10,475,0.95,0.92,289.1,5782,311
70,6,5,15,5,250,1,0.92,288.9,5778,308


In [72]:
for index, row in col1.iterrows():
    print()
    print("____ DATA ",index,"____")

    swarm_size = 70
    pf = 20
    max_iter = 50
    c1 = 2
    mae = 0
    c2 = 2
    posisi = {}
    partikel = {}
    fitness_value = 200

    rentangposisi = {
        "simple": {
            "lower": 5,
            "upper": 7.49
        },
        "average": {
            "lower": 7.5,
            "upper": 12.49
        },
        "complex": {
            "lower": 12.5,
            "upper": 15
        }
    }

    projectactual = {
        'simpleUC': col1.loc[index,"S"],
        'averageUC': col1.loc[index,"A"],
        'complexUC': col1.loc[index,"C"],
        'uaw': col1.loc[index,"UAW"],
        'tcf': col1.loc[index,"TCF"],
        'ecf': col1.loc[index,"ECF"],
        'actualEffort': col1.loc[index,"Real_Effort"]
    }


    def posisiacak():
        xsimple = random.uniform(rentangposisi['simple']['lower'], rentangposisi['simple']['upper'])
        xaverage = random.uniform(rentangposisi['average']['lower'], rentangposisi['average']['upper'])
        xcomplex = random.uniform(rentangposisi['complex']['lower'], rentangposisi['complex']['upper'])
        position = {
                    'simple':xsimple,
                    'average':xaverage,
                    'complex':xcomplex
                    }   
        return position

    def size(xsimple, xaverage, xcomplex):
        ucsimple = xsimple * projectactual['simpleUC']
        ucaverage = xaverage * projectactual['averageUC']
        uccomplex = xcomplex * projectactual['complexUC']
        uucw = ucsimple + ucaverage + uccomplex
        uucp = projectactual['uaw'] + uucw
        return uucp * projectactual['tcf'] * projectactual['ecf']

    def velocity(parameters):
        vsimple = (parameters['w'] * parameters['partikel']['vsimple']) + ((c1 * parameters['r1']) * (parameters['pbest']['simple'] - parameters['partikel']['simple'])) + ((c2 * parameters['r2']) * (parameters['gbest']['simple'] - parameters['partikel']['simple']))

        vaverage = (parameters['w'] * parameters['partikel']['vaverage']) + ((c1 * parameters['r1']) * (parameters['pbest']['average'] - parameters['partikel']['average'])) + ((c2 * parameters['r2']) * (parameters['gbest']['average'] - parameters['partikel']['average']))

        vcomplex = (parameters['w'] * parameters['partikel']['vcomplex']) + ((c1 * parameters['r1']) * (parameters['pbest']['complex'] - parameters['partikel']['complex'])) + ((c2 * parameters['r2']) * (parameters['gbest']['complex'] - parameters['partikel']['complex']))

        return {'vsimple': vsimple, 'vaverage': vaverage, 'vcomplex': vcomplex}

    def inertia(iterasi):
        inertia_max = 0.9
        inertia_min = 0.4
        return inertia_min - (((inertia_max - inertia_min) * iterasi) / max_iter)

    def exceedlimit(value, label):
        if label == 'simple' and value < rentangposisi['simple']['lower']:
            value = rentangposisi['simple']['lower']
        if label == 'simple' and value > rentangposisi['simple']['upper']:
            value = rentangposisi['simple']['upper']
        if label == 'average' and value < rentangposisi['average']['lower']:
            value = rentangposisi['average']['lower']
        if label == 'average' and value > rentangposisi['average']['upper']:
            value = rentangposisi['average']['upper']
        if label == 'complex' and value < rentangposisi['complex']['lower']:
            value = rentangposisi['complex']['lower']
        if label == 'complex' and value > rentangposisi['complex']['upper']:
            value = rentangposisi['complex']['upper']
        return value

    def optimizer():
        partikel = []
        gbests = []
        pbests = []
        ret = []
        for iter in range(max_iter):
            #print('iterasi', iter)
            w = inertia(iter)
            r1 = random.uniform(0, 1)
            r2 = random.uniform(0, 1)

            # initial population
            if iter == 0: 
                for i in range(swarm_size):
                    xsimple = posisiacak()['simple']
                    xaverage = posisiacak()['average']
                    xcomplex = posisiacak()['complex']
                    software_size = size(xsimple,xaverage,xcomplex)
                    estimated_effort = software_size * pf
                    ae = abs(projectactual['actualEffort'] - estimated_effort)
                    vsimple = random.uniform(0, 1)
                    vaverage = random.uniform(0, 1)
                    vcomplex = random.uniform(0, 1)
                    particles = {
                        'simple': xsimple,
                        'average': xaverage,
                        'complex': xcomplex,
                        'size': software_size,
                        'estimated': estimated_effort,
                        'ae': ae,
                        'vsimple':vsimple,
                        'vaverage':vaverage,
                        'vcomplex':vcomplex
                    }
                    partikel.append([])
                    partikel[iter].append(particles)
                    pbests.append([])
                    pbests[iter].append(particles)
                min_ae = min(x['ae'] for x in pbests[iter])
                gbest = next(item for item in pbests[iter] if item['ae'] == min_ae)
                gbests.append([])
                gbests[iter].append(gbest)
                #print(gbests[iter])

            if iter > 0:
                for count,value in enumerate(partikel[iter-1]):
                    parameters = {
                        'pbest': pbests[iter-1][count],
                        'partikel': value,
                        'gbest': gbests[iter-1][0],
                        'w':w,
                        'r1':r1,
                        'r2':r2
                    }
                    vel = velocity(parameters)
                    xsimple = value['simple'] + vel['vsimple']
                    xaverage = value['average'] + vel['vaverage']
                    xcomplex = value['complex'] + vel['vcomplex']
                    xsimple = exceedlimit(xsimple, 'simple')
                    xaverage = exceedlimit(xaverage, 'average')
                    xcomplex = exceedlimit(xcomplex, 'complex')
                    software_size = size(xsimple,xaverage,xcomplex)
                    estimated_effort = software_size * pf
                    ae = abs(projectactual['actualEffort'] - estimated_effort)
                    particles = {
                        'simple': xsimple,
                        'average': xaverage,
                        'complex': xcomplex,
                        'size': software_size,
                        'estimated': estimated_effort,
                        'ae': ae,
                        'vsimple':vel['vsimple'],
                        'vaverage':vel['vaverage'],
                        'vcomplex':vel['vcomplex']
                    }
                    partikel.append([])
                    partikel[iter].append(particles)

                    if pbests[iter-1][count]['ae'] > partikel[iter][count]['ae']:
                       pbests[iter].append(partikel[iter][count])
                    else:
                        pbests[iter].append(partikel[iter-1][count])

                min_ae = min(x['ae'] for x in pbests[iter])
                gbest = next(item for item in pbests[iter] if item['ae'] == min_ae)
                gbests.append([])
                gbests[iter].append(gbest)
                #print(gbests[iter])

                if gbests[iter][0]['ae'] < fitness_value:
                    return gbests[iter][0]
                ret.append(gbests[iter][0])
        return ret

    results = optimizer()
    
    #print(projectactual)
    #print('results ',results)
    if len(results) == 9:
        #mae = mae + results['ae']
        print(results)
    else:
        min_ae = min(x['ae'] for x in results)
        result = next(item for item in results if item['ae'] == min_ae)
        #mae = mae + result['ae']
        print(result)
    #print(mae)


____ DATA  1 ____
{'simple': 7.49, 'average': 12.49, 'complex': 15, 'size': 274.772736, 'estimated': 5495.45472, 'ae': 2474.5452800000003, 'vsimple': 0.22146877505755855, 'vaverage': 2.009327343692568, 'vcomplex': 0.3283083397873672}

____ DATA  2 ____
{'simple': 6.54954578399063, 'average': 8.540313104385168, 'complex': 13.412767232532762, 'size': 398.11265423279536, 'estimated': 7962.2530846559075, 'ae': 0.2530846559075144, 'vsimple': 0.10242389838639254, 'vaverage': 0.15032034751634105, 'vcomplex': 0.28520010899700843}

____ DATA  3 ____
{'simple': 7.49, 'average': 12.49, 'complex': 15, 'size': 312.24656000000004, 'estimated': 6244.931200000001, 'ae': 1690.068799999999, 'vsimple': 1.1535286340780906, 'vaverage': 0.7575194032599621, 'vcomplex': 2.0248199859951104}

____ DATA  4 ____
{'simple': 7.49, 'average': 12.49, 'complex': 15, 'size': 323.79165000000006, 'estimated': 6475.833000000001, 'ae': 1329.1669999999986, 'vsimple': 1.669611910638086, 'vaverage': 4.414259426768132, 'vcomp

{'simple': 7.055721527486583, 'average': 12.49, 'complex': 15.0, 'size': 201.14352960523723, 'estimated': 4022.870592104745, 'ae': 2393.129407895255, 'vsimple': 3.0160503539785126e-13, 'vaverage': 0.0, 'vcomplex': 0.0}

____ DATA  36 ____
{'simple': 5.896356433917005, 'average': 12.49, 'complex': 15, 'size': 319.146168, 'estimated': 6382.92336, 'ae': 29.076640000000225, 'vsimple': 0.4946794943195607, 'vaverage': 0.2303590798612333, 'vcomplex': 0.5020626934635551}

____ DATA  37 ____
{'simple': 5.499565861039148, 'average': 11.678790479966723, 'complex': 12.984519451127609, 'size': 320.15579689567903, 'estimated': 6403.115937913581, 'ae': 3.1159379135806375, 'vsimple': 0.45579296942534514, 'vaverage': -0.4914451275425085, 'vcomplex': -0.6826079470940023}

____ DATA  38 ____
{'simple': 7.34990710905817, 'average': 12.49, 'complex': 15.0, 'size': 165.46722685462484, 'estimated': 3309.344537092497, 'ae': 3050.655462907503, 'vsimple': 2.1345190439187333e-13, 'vaverage': 0.0, 'vcomplex': 0.0